## MNIST + Convolutional Neural Network

In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.datasets import mnist
from keras.utils import to_categorical

In [ ]:
img_rows = img_cols = 28
num_classes = 10

(x_train, y_train),(x_test, y_test) = mnist.load_data()

# Reshape from (60000, 28,28) to (60000,28,28,1) : 60000 samples, 28x28 pixels, 1 channel (B/W)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
# Reshape from (10000, 28,28) to (10000,28,28,1) : 10000 samples, 28x28 pixels, 1 channel (B/W)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

In [ ]:
print (x_train.shape)
print (y_train.shape)

In [ ]:
# Normalize pixel values: [0-255] --> [0.0-1.0]
x_train, x_test = x_train / 255.0, x_test / 255.0

# One-hot encode labels
# We could skip this step and use the 'sparse_categorical_crossentropy' loss function instead
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [ ]:
print (y_train[:1])

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])

print(model.summary())

In [ ]:
batch_size = 128
epochs = 10

In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=batch_size, epochs=epochs)

In [ ]:
score = model.evaluate(x_test, y_test)
print(score)

In [ ]:
model.save("fc-mnist")

## Predict home-made digits

In [ ]:
import numpy as np
import cv2

np.set_printoptions(precision=2, suppress=True)

def loadImage(filename):
        img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
        img = img / 255
        # Reshape from (28,28) to (1,28,28,1) : 1 sample, 28x28 pixels, 1 channel (B/W)
        img = np.expand_dims(img, axis=0)
        img = np.expand_dims(img, axis=0)
        img = np.reshape(img, (1,img_cols,img_rows,1))
        return np.array(img)

In [ ]:
from keras.models import load_model
model = load_model("cnn-mnist")
print(model.summary())

In [ ]:
for digit in range(0,10):
    img = "digits/%s.png" % digit
    img = loadImage(img)
    classes = model.predict(img)
    predicted = np.argmax(classes)
    result = ''
    if (predicted != digit):
        result = "WRONG!"
    print(" *%s* %s %s" % (digit, classes, result))